In [5]:
import aiohttp
import asyncio
import bs4
import datetime
from io import BytesIO
import pandas as pd
import PyPDF2
import requests
import time
from tqdm.contrib.telegram import tqdm
import os

In [6]:
TOKEN = os.getenv('TOKEN')
CHAT_ID = os.getenv('CHAT_ID')

In [7]:
railway_news = "https://railway.uz/uz/informatsionnaya_sluzhba/novosti/?PAGEN_1={page}" #182
# uzbekistan_news = "https://uzlidep.uz/uz/news-of-uzbekistan?page={page}" #149

LAST = 182
# party_last = 1250

In [8]:
BASE_URL = "https://railway.uz"

In [19]:

# scraper.py
import requests
from bs4 import BeautifulSoup

url = 'https://uzlidep.uz/uz/news-of-uzbekistan?page=2'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')
quotes = soup.find('div', class_='publications list').find_all('a')

for i in quotes:
    print(i['href'])

# print(quotes)
   

/uz/news-of-uzbekistan/12984
/uz/news-of-uzbekistan/12984
/uz/news-of-uzbekistan/12936
/uz/news-of-uzbekistan/12936
/uz/news-of-uzbekistan/12892
/uz/news-of-uzbekistan/12892
/uz/news-of-uzbekistan/12776
/uz/news-of-uzbekistan/12776
/uz/news-of-uzbekistan/12718
/uz/news-of-uzbekistan/12718
/uz/news-of-uzbekistan/12684
/uz/news-of-uzbekistan/12684
/uz/news-of-uzbekistan/12665
/uz/news-of-uzbekistan/12665
/uz/news-of-uzbekistan/12641
/uz/news-of-uzbekistan/12641
/uz/news-of-uzbekistan/12462
/uz/news-of-uzbekistan/12462
/uz/news-of-uzbekistan/12436
/uz/news-of-uzbekistan/12436


In [9]:
links = []
async def main():
    async with aiohttp.ClientSession() as session:
        for page in tqdm(range(1, LAST+1), token=TOKEN, chat_id=CHAT_ID):
            try:
                async with session.get(railway_news.format(page=page)) as resp:
                    if resp.status == 200:
                        data = await resp.text()
                        soup = bs4.BeautifulSoup(data)
                        selected_links = soup.find('ul', class_='dlist news-list news-list--inner').find_all('a')
                        links.extend([BASE_URL+link['href'] for link in selected_links])
                    elif resp.status != 404:
                        await asyncio.sleep(60) # 1 minute
            except Exception as e:
                print(e)
await main()

  0%|          | 0/182 [00:00<?, ?it/s]

In [16]:
links

['https://railway.uz/uz/informatsionnaya_sluzhba/novosti/34476/',
 'https://railway.uz/uz/informatsionnaya_sluzhba/novosti/34471/',
 'https://railway.uz/uz/informatsionnaya_sluzhba/novosti/34457/',
 'https://railway.uz/uz/informatsionnaya_sluzhba/novosti/34453/',
 'https://railway.uz/uz/informatsionnaya_sluzhba/novosti/34441/',
 'https://railway.uz/uz/informatsionnaya_sluzhba/novosti/34438/',
 'https://railway.uz/uz/informatsionnaya_sluzhba/novosti/34357/',
 'https://railway.uz/uz/informatsionnaya_sluzhba/novosti/34353/',
 'https://railway.uz/uz/informatsionnaya_sluzhba/novosti/34345/',
 'https://railway.uz/uz/informatsionnaya_sluzhba/novosti/34337/',
 'https://railway.uz/uz/informatsionnaya_sluzhba/novosti/34322/',
 'https://railway.uz/uz/informatsionnaya_sluzhba/novosti/34314/',
 'https://railway.uz/uz/informatsionnaya_sluzhba/novosti/34304/',
 'https://railway.uz/uz/informatsionnaya_sluzhba/novosti/34294/',
 'https://railway.uz/uz/informatsionnaya_sluzhba/novosti/34280/',
 'https://

In [ ]:
results = []
async def main():
    async with aiohttp.ClientSession() as session:
        for link in tqdm(links, token=TOKEN, chat_id=CHAT_ID):
            try:
                async with session.get(link, ssl=False) as resp:
                    if resp.status == 200:
                        data = await resp.text()
                        soup = bs4.BeautifulSoup(data)
                        content = soup.find('div', class_="inner-content white-inner-section").find_all("p")
                        title = soup.find('h3', class_="inner-content__title").text
                        p_text = [p.text for p in content]
                        
                        results.append({
                            "title": title.strip(),
                            "content": " ".join(p_text)
                        })
                        
                        print(title, p_text)
                    elif resp.status != 404:
                        await asyncio.sleep(60) # 1 minute
            except Exception as e:
                print(link,e)
await main()

  0%|          | 0/1001 [00:00<?, ?it/s]

Jamiyat rahbariyatining tashrifi davom etmoqda ['\nAvval xabar berganimizdek, "O\'zbekiston temir yo\'llari" aksiyadorlik jamiyati boshqaruvi raisi vazifasini bajaruvchi Z.Narzullayev rahbarligidagi Ishchi guruh Surxondaryo viloyatiga tashrifini davom ettirmoqda. Ishchi guruhning navbatdagi manzili "Termiz mintaqaviy temir yo\'l uzel" unitar korxonasi bo\'ldi.\n\r\n\t Z.Narzullayev Termiz MTU va liniya korxonalari rahbarlari bilan yig\'ilish o\'tkazdi. Unda joriy jarayonlarni optimallashtirish va korxonalar samaradorligini oshirish zarurligiga alohida e\'tibor qaratdi.\xa0\n\r\n\t Shu bilan birga, Z.Narzullayev MTU Termiz tasarrufidagi vokzallarga tashrif buyurdi, yo\'lovchilar uchun yaratilgan sharoitlar bilan tanishdi.\n\r\n\t Bugungi kunda yo’lovchilarga xizmat ko’rsatish sifati qoniqarsiz holatda, muammolar yetarlicha, – deya qayd etdi Z.Narzullayev. – Buni yaxshilashni ertaga emas, balki bugundan boshlashimiz shart.\n\r\n\t Ishchi guruhning viloyatlardagi tashrifi davom etmoqda\r\

In [20]:
df = pd.DataFrame(results)

In [21]:
df

""


In [22]:
results

[]

In [24]:
df.to_pickle("Railway.pickle")